In [1]:
from keras.applications import VGG16

In [40]:
vgg16_conv = VGG16(weights='imagenet', include_top=False,input_shape=(224, 224, 3) )

In [41]:
vgg16_conv.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [79]:
for layer in vgg16_conv.layers:
    layer.trainable = False
for layer in vgg16_conv.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x00000162834B1B00> False
<keras.layers.convolutional.Conv2D object at 0x0000016283499AC8> False
<keras.layers.convolutional.Conv2D object at 0x0000016283475C50> False
<keras.layers.pooling.MaxPooling2D object at 0x00000162834DA198> False
<keras.layers.convolutional.Conv2D object at 0x00000162834DAD30> False
<keras.layers.convolutional.Conv2D object at 0x00000162829EC4A8> False
<keras.layers.pooling.MaxPooling2D object at 0x0000016282A0B828> False
<keras.layers.convolutional.Conv2D object at 0x0000016282A0B6A0> False
<keras.layers.convolutional.Conv2D object at 0x0000016282A46128> False
<keras.layers.convolutional.Conv2D object at 0x0000016282A62208> False
<keras.layers.pooling.MaxPooling2D object at 0x0000016282A85828> False
<keras.layers.convolutional.Conv2D object at 0x0000016282A856A0> False
<keras.layers.convolutional.Conv2D object at 0x0000016282ABF128> False
<keras.layers.convolutional.Conv2D object at 0x0000016282ADD1D0> False
<ker

In [80]:
from keras import models, layers, optimizers

In [82]:
model = models.Sequential()
model.add(vgg16_conv)
model.add(layers.Flatten())
model.add(layers.Dense(1024,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10,activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_4 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                10250     
Total params: 40,416,074
Trainable params: 25,701,386
Non-trainable params: 14,714,688
_________________________________________________________________


In [70]:
import cv2
import numpy as np
from keras.datasets import cifar10
from keras import backend as K
from keras.utils import np_utils

nb_train_samples = 5000 # 5000 training samples
nb_valid_samples = 1000 # 1000 validation samples
num_classes = 10

def load_cifar10_data(img_rows, img_cols):

    # Load cifar10 training and validation sets
    (X_train, Y_train), (X_valid, Y_valid) = cifar10.load_data()

    # Resize trainging images
    if K.image_dim_ordering() == 'th':
        X_train = np.array([cv2.resize(img.transpose(1,2,0), (img_rows,img_cols)).transpose(2,0,1) for img in X_train[:nb_train_samples,:,:,:]])
        X_valid = np.array([cv2.resize(img.transpose(1,2,0), (img_rows,img_cols)).transpose(2,0,1) for img in X_valid[:nb_valid_samples,:,:,:]])
    else:
        X_train = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_train[:nb_train_samples,:,:,:]])
        X_valid = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_valid[:nb_valid_samples,:,:,:]])

    # Transform targets to keras compatible format
    Y_train = np_utils.to_categorical(Y_train[:nb_train_samples], num_classes)
    Y_valid = np_utils.to_categorical(Y_valid[:nb_valid_samples], num_classes)

    return X_train, Y_train, X_valid, Y_valid

In [71]:
X_train, Y_train, X_valid, Y_valid = load_cifar10_data(224,224)

In [75]:
print(X_train.shape, X_valid.shape)

(5000, 224, 224, 3) (1000, 224, 224, 3)


In [76]:
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

In [84]:
model.compile(optimizer=Adam(lr=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(X_train,Y_train,batch_size=25,epochs=50,validation_split=0.1,verbose=2)

Train on 4500 samples, validate on 500 samples
Epoch 1/50
